# Prepare custom dataset

In [ ]:
# Copyright 2021 wngfra.
# SPDX-License-Identifier: Apache-2.0

import glob
import os
import re
import numpy as np
from bidict import bidict
from torch.utils.data import Dataset
from torchvision import transforms

class Texture:
    """ Create a bidict from a texture name list."""

    def __init__(self, texture_names):
        self.texture_by_id = bidict()
        for i, tn in enumerate(set(texture_names)):
            self.texture_by_id[tn] = i

    def get_id(self, texture_name: str):
        return self.texture_by_id[texture_name]

    def get_name(self, texture_id: int):
        return self.texture_by_id.inverse[texture_id]


class TacDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.filelist = [y for x in os.walk(
            root_dir) for y in glob.glob(os.path.join(x[0], '*.npy'))]
        self.params = [(0.0, 0.0)] * len(self.filelist)
        self.texture_names = []
        for i, filename in enumerate(self.filelist):
            basename = os.path.basename(filename)
            namegroups = basename.split('_')

            self.texture_names.append(namegroups[0])
            self.params[i] = [float(re.search(r"\d+.\d+", namegroups[1]).group(0)),
                              float(re.search(r"\d+.\d+", namegroups[2]).group(0))]
        self.textures = Texture(self.texture_names)

    def __len__(self):
        return len(self.filelist)

    def __getitem__(self, index):
        filename = os.path.join(self.root_dir, self.filelist[index])
        rawdata  = np.load(filename)
        tacdata  = rawdata[:, :16]
        wrench   = rawdata[:, -6:]
        texture_name = self.texture_names[index]
        if self.transform:
            tacdata = self.transform(tacdata)
        return np.hstack([tacdata, wrench]), self.params[index], self.textures.get_id(texture_name) 

    def get_texture_name(self, texture_id):
        return self.textures.get_name(texture_id)

    
""" Custom transforms """

class Normalize(object):
    def __init__(self, axis=0):
        self.axis = axis

    def __call__(self, sample):
        return (sample - np.mean(sample, axis=self.axis)) / np.std(sample, axis=self.axis)


class ToFourierBasis(object):
    """ Transform a sample to its Fourier basis coefficients """
    def __init__(self, n_basis, period) -> None:
        self.basis = basis.Fourier(
            [0, 2 * np.pi], n_basis=n_basis, period=period)

    def __call__(self, sample):
        # Input sample matrix
        # Each row contains one observation and each column represents one variable
        fd = FDataGrid(sample.T).to_basis(self.basis)
        coeffs = fd.coefficients.squeeze()

        return coeffs.T

transform = transforms.Compose([Normalize(axis=0)])
ds = TacDataset('../data', transform=transform)

# Group all samples and labels, save to `.mat` file

In [ ]:
from scipy.io import savemat

mdata = []
mlabels = []

for i, (sample, param, label) in enumerate(ds):
    mdata.append(sample[:, :16])
    mlabels.append(label)
    
mdata = np.asarray(mdata, dtype=object)

tacmat = {"mdata": mdata, "labels": mlabels}
savemat("TacMat", tacmat)

# Visualize raw data in time and frequency domains

In [ ]:
import matplotlib.pyplot as plt
from numpy.fft import fft

plt.rcParams['figure.dpi'] = 150

def plot2(item):
    data, param, label = item[0], item[1], item[2]
    data = data[:, :16]
    L = data.shape[0]
    Y = np.fft.fft(data, axis=0)
    Ys = np.abs(Y / L)
    Ys = Ys[1:L//2+1, :]
    
    plt.figure()
    plt.subplot(211)
    plt.plot(data)
    plt.subplot(212)
    plt.plot(Ys[:64, :])
    title = ds.get_texture_name(label)
    plt.suptitle("{} @ {}N and {}mmps".format(title, param[0], param[1]))
    plt.show()

for item in ds:
    plot2(item)

# Compress data with Tucker decomposition
1. Compute covariance matrix for each multi-channel frequency series
2. Stack covariance matrix into a 3D covariance tensor $T \in \mathbb{R}^{C \times C \times N}$
3. Use core tensor $\mathcal{G}$ of the Tucker decomposition $T = \mathcal{G} \times_1 U_1 \times_2 U_2 \times_3 U_3$ as a compressed representation

In [ ]:
import pandas as pd
from numpy.fft import fft
from skfda import FDataGrid
from skfda.representation import basis
from tensorly.decomposition import non_negative_tucker

N_BASIS = 33
STRIDE = 128
WINDOW_SIZE = 256

fd_basis = basis.Fourier([0, 2 * np.pi], n_basis=N_BASIS, period=1)

def compute_cov_fda(data):
    ''' Compute covariance matrix with functional basis decomposition.'''
    fd = FDataGrid(data.T).to_basis(fd_basis)
    coeffs = fd.coefficients.squeeze()
    return np.cov(coeffs.T)

def compute_cov_fft(data):
    L = data.shape[0]
    Y = fft(data, axis=0)
    Ys = np.abs(Y / L)
    return np.cov(Ys[1:L//2+1, :])

cov_array = []
labels = []

for i, (sample, param, label) in enumerate(ds):
    cov = compute_cov_fda(sample)
    cov_array.append(cov)
    labels.append(ds.get_texture_name(label))

cov_tensor = np.transpose(np.asarray(cov_array), [1, 2, 0])
core, factors = non_negative_tucker(cov_tensor, rank=(3, 1, cov_tensor.shape[2]))
core3d = core.squeeze().T  # get projected vectors

df0 = pd.DataFrame(labels, columns=["texture"])
df1 = pd.DataFrame(core3d, columns=["x1", "x2", "x3"])
df  = pd.concat([df0, df1], axis=1)

# Visualize compressed core tensor

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

shift = None
df_ = df.iloc[3:, :]

textures = df_["texture"].unique()
cmap = plt.cm.get_cmap("plasma", len(textures))

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")
for i, texture in enumerate(textures):
    X = df_.loc[df_["texture"] == texture]
    # plot core vectors
    xs, ys, zs = X["x1"], X["x2"], X["x3"]
    if shift is not None:
        xs, ys, zs = shift(xs), shift(ys), shift(zs)
    ax.scatter(xs, ys, zs, s=20, c=np.tile(cmap(i), (len(xs), 1)))
ax.legend(textures)
plt.show()

# Construct RNN-AutoEncoder (RAE)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence


class Encoder(nn.Module):
    """ Recurent Variational Autoencoder """
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, device, dropout=0.3):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.device = device
        
        self.rnn = nn.GRU(input_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
        self.fc_mu = nn.Linear(hidden_dim, output_dim)
        self.fc_var = nn.Linear(hidden_dim, output_dim)
        
        self.to(device)
        
    def forward(self, x):
        packed_in = pack_padded_sequence(x[0].to(self.device), x[1].cpu().numpy(), batch_first=True)
        rnn_out, self.hidden = self.rnn(packed_in)
        x_in = self.hidden[-1].squeeze()
        mu = self.fc_mu(x_in)
        var = self.fc_var(x_in)
        
        return mu
    
    def init_hidden(self, batch_dim):
        return (torch.zeros(self.n_layers, batch_dim, self.hidden_dim, device=self.device),
                torch.zeros(self.n_layers, batch_dim, self.hidden_dim, device=self.device))


class RVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, encoding_dim, extra_dim, output_dim, n_layers, device):
        super(RVAE, self).__init__()
        self.device = device
        
        self.encoder = Encoder(input_dim, hidden_dim, encoding_dim, n_layers, device)
        self.classifier = nn.Sequential(
            nn.Linear(encoding_dim + extra_dim, 4 * (encoding_dim + extra_dim)),
            nn.ReLU(),
            nn.Linear(4 * (encoding_dim + extra_dim), output_dim),
            nn.Sigmoid()
        ).to(device)

    def forward(self, x):
        encoded = self.encode(x[:2])
        x_in = torch.hstack([encoded, x[2].to(self.device)])
        y = self.classifier(x_in)

        return y

    def encode(self, x):
        return self.encoder(x)
    
""" Custom collate functions"""

class PadSequence(object):
    def __call__(self, batch):
        # Each element in "batch" is a tuple (data, label).
        # Sort the batch in the descending order
        sorted_batch = sorted(batch, key=lambda x: x[0].shape[0], reverse=True)
        # Get each sequence and pad it
        sequences = [torch.tensor(x[0], dtype=torch.float) for x in sorted_batch]
        sequences_padded = pad_sequence(
            sequences, batch_first=True)
        # Store the length of each sequence
        lengths = torch.tensor([len(x) for x in sequences])
        params = torch.tensor(list(map(lambda x: x[1], sorted_batch)), dtype=torch.float)
        labels = torch.tensor(list(map(lambda x: x[2], sorted_batch)))
        return sequences_padded, lengths, params, labels

# Train the network

In [ ]:
import matplotlib.pyplot as plt
import torch
from torch import optim
from torch.utils.data import DataLoader

BATCH_SIZE = 4
EPOCHS = 100
INPUT_DIM = 22

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

train_loader = DataLoader(ds, batch_size=BATCH_SIZE, collate_fn=PadSequence(), num_workers=6, shuffle=True)
rvae = RVAE(input_dim=INPUT_DIM, hidden_dim=32, encoding_dim=3, extra_dim=2, output_dim=4, n_layers=2, device=device)

def train_once(x, y, model, optimizer, criterion):
    optimizer.zero_grad()
    output = model(x)
    target = y.to(device)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    
    return loss.item()
    
def train_model(data_loader, model):
    optimizer = optim.SGD(model.parameters(), lr = 1e-3)
    criterion = nn.CrossEntropyLoss()
    loss_list = []
    
    for epoch in range(EPOCHS):
        running_loss = 0.0
        
        for i, (batch, lengths, params, targets) in enumerate(data_loader):
            loss = train_once((batch, lengths, params), targets, model, optimizer, criterion)
            running_loss += loss
            loss_list.append(loss)
            
            if i % 10 == 9:    # print every 2000 mini-batches
                print('Epoch {}, {:.2f}% - loss: {:.6f}'.format(epoch + 1, 100.0 * (i + 1.0) / len(data_loader), running_loss / 10))
                running_loss = 0.0
                
    print("Training finished.")
    plt.plot(loss_list)
    plt.show()
    
train_model(train_loader, rvae)